In [11]:
import numpy as np
import scipy.fftpack as fourier
import matplotlib
import matplotlib.pyplot as plt
import scipy.io.wavfile as waves
import pyaudio as pa 
import struct 
import statsmodels.api as sm
from scipy.signal import find_peaks

In [13]:
#Definicion de la primera frecuencia de cada nota musical y del array con los nombre de las notas.

hzInicialFa = 21.826            
hzInicialFaS = 23.125           
hzInicialSol = 24.50            
hzInicialSolS = 25.96
hzInicialLa = 27.50
hzInicialLaS = 29.14
hzInicialSi = 30.87
hzInicialDo = 32.7              
hzInicialDoS = 34.65
hzInicialRe = 36.71
hzInicialReS = 38.89
hzInicialMi = 41.2

hzIniciales = np.array([hzInicialFa, hzInicialFaS, hzInicialSol, hzInicialSolS, hzInicialLa, hzInicialLaS, hzInicialSi, hzInicialDo, hzInicialDoS, hzInicialRe, hzInicialReS, hzInicialMi], dtype=float)

nombreNota = np.array(['Fa','Fa#','Sol','Sol#','La','La#','Si','Do','Do#','Re','Re#','Mi'], dtype = str)

In [16]:
#Funciones para conseguir los 0.5 anteriores y los 0.5 seguientes de un Hz

def nAnteriores(Hz):
    vAnteriores = np.array([], dtype=float)
    for i in range(5):
        valor = 0.5 - (i/10)
        vAnteriores = np.append(vAnteriores, (Hz - valor))
    return vAnteriores

def nSiguientes(Hz):
    vSiguientes = np.array([], dtype=float)
    for i in range(5):
        valor = (i/10) + 0.1
        vSiguientes = np.append(vSiguientes, (Hz + valor))
    return vSiguientes

#Funciones para acumular los valores anteiores y siguientes.

def vAnteriores(Hz):
    valoresAnteriores = np.array([], dtype=float)
    valoresAnteriores = np.append(valoresAnteriores, nAnteriores(Hz))
    valoresAnteriores = np.append(valoresAnteriores, Hz)
    return valoresAnteriores

def vSiguientes(Hz):
    valoresSiguientes = np.array([], dtype=float)
    valoresSiguientes = np.append(valoresSiguientes, nSiguientes(Hz))
    return valoresSiguientes

#Funcion para el guardado automatico de las frecuencias en un array.

def LlenadoAutomaticoDeHz(Hz):
    allHz = np.array([], dtype=float)
    allHz = np.append(allHz, vAnteriores(Hz))
    allHz = np.append(allHz, vSiguientes(Hz))
    #print(allHz)
    return allHz

In [19]:
#Frecuencias de Fa, Fa# y Sol
FaFaSSolHz = np.array([], dtype=float)
for i in range(3):
    for j in range(9):
        Hz2 = hzIniciales[i] * (2 ** j)
        FaFaSSolHz = np.append(FaFaSSolHz, LlenadoAutomaticoDeHz(Hz2))

#Frecuencias de Sol#, La, La#, Si, Do, Do#, Re, Re#, Mi
allHz = np.array([], dtype=float)
for i in range(3, 12):
    for j in range(8):
        Hz2 = hzIniciales[i] * (2 ** j)
        allHz = np.append(allHz, LlenadoAutomaticoDeHz(Hz2))


In [20]:
#Ingreso de las frecuencias en los array de las notas Fa, Fa#, Sol
# 99 valores en total para cada nota.

total = 99

FaHz = np.array([], dtype=float)
for i in range(total):
    FaHz = np.append(FaHz, FaFaSSolHz[i])

FaSHz = np.array([], dtype=float)
for i in range(total, total*2):
    FaSHz = np.append(FaSHz, FaFaSSolHz[i])

SolHz = np.array([], dtype=float)
for i in range(total*2, total*3):
    SolHz = np.append(SolHz, FaFaSSolHz[i])

#Ingreso de las frecuencias en los array de las notas Sol#, La, La#, Si, Do, Do#, Re, Re#, Mi
# 88 valores en total para cada nota.

total2 = 88

SolSHz = np.array([], dtype=float)
for i in range(total2):
    SolSHz = np.append(SolSHz, allHz[i])

LaHz = np.array([], dtype=float)
for i in range(total2, total2*2):
    LaHz = np.append(LaHz, allHz[i])

LaSHz = np.array([], dtype=float)
for i in range(total2*2, total2*3):
    LaSHz = np.append(LaSHz, allHz[i])

SiHz = np.array([], dtype=float)
for i in range(total2*3, total2*4):
    SiHz = np.append(SiHz, allHz[i])

DoHz = np.array([], dtype=float)
for i in range(total2*4, total2*5):
    DoHz = np.append(DoHz, allHz[i])

DoSHz = np.array([], dtype=float)
for i in range(total2*5, total2*6):
    DoSHz = np.append(DoSHz, allHz[i])

ReHz = np.array([], dtype=float)
for i in range(total2*6, total2*7):
    ReHz = np.append(ReHz, allHz[i])

ReSHz = np.array([], dtype=float)
for i in range(total2*7, total2*8):
    ReSHz = np.append(ReSHz, allHz[i])

MiHz = np.array([], dtype=float)
for i in range(total2*8, total2*9):
    MiHz = np.append(MiHz, allHz[i])

In [21]:
#.---.HACIA ARRIBA ES LA DEFINICION DE LOS HZ Y SU ALMACENAMIENTO EN SUS RESPECTIVAS NOTAS MUSICALES.---.#

In [ ]:
matplotlib.use('TkAgg')

FRAMES = 1024*8                                   # Tamaño del paquete a procesar
FORMAT = pa.paInt16                               # Formato de lectura INT 16 bits
CHANNELS = 1
Fs = 44100                                        # Frecuencia de muestreo típica para audio

p = pa.PyAudio()

stream = p.open(                                  # Abrimos el canal de audio con los parámeteros de configuración
    format = FORMAT,
    channels = CHANNELS,
    rate = Fs,
    input=True,
    output=True,
    frames_per_buffer=FRAMES
)

# Creamos una gráfica con 2 subplots y configuramos los ejes

fig, (ax,ax1) = plt.subplots(2)

x_audio = np.arange(0,FRAMES,1)
x_fft = np.linspace(0, Fs, FRAMES)
line, = ax.plot(x_audio, np.random.rand(FRAMES),'r')
line_fft, = ax1.semilogx(x_fft, np.random.rand(FRAMES), 'b')

ax.set_ylim(-32500,32500)
ax.ser_xlim = (0,FRAMES)
Fmin = 1
Fmax = 5000
ax1.set_xlim(Fmin,Fmax)

fig.show()


F = (Fs/FRAMES)*np.arange(0,FRAMES//2)                 # Creamos el vector de frecuencia para encontrar la frecuencia dominante

while True:

    data = stream.read(FRAMES)                         # Leemos paquetes de longitud FRAMES
    dataInt = struct.unpack(str(FRAMES) + 'h', data)   # Convertimos los datos que se encuentran empaquetados en bytes

    line.set_ydata(dataInt)                            # Asignamos los datos a la curva de la variación temporal

    M_gk = abs(fourier.fft(dataInt)/FRAMES)            # Calculamos la FFT y la Magnitud de la FFT del paqute de datos

    ax1.set_ylim(0,np.max(M_gk+10)) 
    line_fft.set_ydata(M_gk)                           # Asigmanos la Magnitud de la FFT a la curva del espectro 

    M_gk = M_gk[0:FRAMES//2]                           # Tomamos la mitad del espectro para encontrar la Frecuencia Dominante
    Posm = np.where(M_gk == np.max(M_gk))
    HzMaximo = F[Posm]                                   # Encontramos la frecuencia que corresponde con el máximo de M_gk

    determinarNotaMusical(HzMaximo)

    fig.canvas.draw()
    fig.canvas.flush_events()